# Setup

In [1]:
import boto3
import pandas as pd
import io
import tqdm

# Compare CoralNET sources in pyspacer with new images

In [2]:
s3 = boto3.client('s3')
bucket_name = '2310-coralnet-public-sources'

response = s3.list_objects_v2(
    Bucket=bucket_name,
    Delimiter='/'
)

# Get common prefixes (folders)
if 'CommonPrefixes' in response:
    folders = [prefix['Prefix'] for prefix in response['CommonPrefixes']]
    print(f"{len(folders)} Folders found:")
else:
    print("No folders found in the bucket")

580 Folders found:


In [3]:
len(folders)

580

In [4]:
labelset = set()
data = []
for source in tqdm.tqdm(folders):
    if not source.startswith('s'):
        print(source)

    file_key = f'{source}annotations.csv'

    # Get the file from S3
    try:
        s3.head_object(Bucket=bucket_name, Key=file_key)
    except s3.exceptions.ClientError as e:
        if e.response['Error']['Code'] == '404':
            print(f"File {file_key} not found in bucket")
            continue
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    row = [source, len(annotations_df), annotations_df['Image ID'].nunique(), annotations_df['Label ID'].nunique()]
    data.append(row)
    labelset = labelset | set(annotations_df['Label ID'].unique())

  0%|          | 2/580 [00:00<00:38, 15.12it/s]

coralnet-public-images/
File coralnet-public-images/annotations.csv not found in bucket


 37%|███▋      | 213/580 [00:18<00:28, 12.96it/s]

File s295/annotations.csv not found in bucket


 49%|████▉     | 286/580 [00:23<00:15, 19.20it/s]

...

 54%|█████▍    | 314/580 [00:25<00:21, 12.40it/s]

File s3363/annotations.csv not found in bucket


 71%|███████▏  | 414/580 [00:36<00:12, 13.77it/s]

File s372/annotations.csv not found in bucket


100%|██████████| 580/580 [00:49<00:00, 11.73it/s]


In [5]:
print(f"The total number of unique labels is {len(labelset)}")

The total number of unique labels is 2245


In [6]:
coralnet_summary_df = pd.DataFrame(data, columns=['Source', 'Num Annotations', 'Num Images', 'Num Unique Labels'])
coralnet_summary_df

,Source,Num Annotations,Num Images,Num Unique Labels
0,s1073/,45000,225,27
1,s1076/,41800,209,28
2,s109/,197100,3942,78
3,s1097/,16860,1686,78
4,s1162/,25,5,4
...,...,...,...,...
569,s841/,58200,3880,75
570,s842/,30309,940,3
571,s843/,56400,2820,64
572,s921/,1860,62,27


In [7]:
print("There are a total of %d annotations across %d images (average %d annotations per image) and %d unique labels." %
      (coralnet_summary_df["Num Annotations"].sum(), coralnet_summary_df["Num Images"].sum(), 
       coralnet_summary_df["Num Annotations"].sum() / coralnet_summary_df["Num Images"].sum(), len(labelset)))

There are a total of 11066639 annotations across 421642 images (average 26 annotations per image) and 2245 unique labels.


In [8]:
s3 = boto3.client('s3')
bucket_name = 'dev-datamermaid-sm-sources'

response = s3.list_objects_v2(
    Bucket=bucket_name,
    Delimiter='/'
)
# Get common prefixes (folders)
if 'CommonPrefixes' in response:
    folders_new = [prefix['Prefix'] for prefix in response['CommonPrefixes']]
    folder = 'coralnet-public-images/'
    # List subfolders within coralnet-public-images
    paginator = s3.get_paginator('list_objects_v2')
    sub_response = {'CommonPrefixes': []}
    for page in paginator.paginate(Bucket=bucket_name, Prefix=folder, Delimiter='/'):
        if 'CommonPrefixes' in page:
            sub_response['CommonPrefixes'].extend(page['CommonPrefixes'])
    if 'CommonPrefixes' in sub_response:
        print("Subfolders in coralnet-public-images/:")
        folders_new = [prefix['Prefix'] for prefix in sub_response['CommonPrefixes']]
        folders_new = [folder.replace("coralnet-public-images/", "") for folder in folders_new]
        # for subfolder in sub_response['CommonPrefixes']:
        #     print(f"- {subfolder['Prefix']}")
    else:
        print("No subfolders found in coralnet-public-images/")
else:
    print("No folders found in the bucket")

Subfolders in coralnet-public-images/:


In [9]:
labelset2 = set()
data2 = []
for source in tqdm.tqdm(folders_new):
    if not source.startswith('s'):
        print(source)

    file_key = f'coralnet-public-images/{source}annotations.csv'

    # Get the file from S3
    try:
        s3.head_object(Bucket=bucket_name, Key=file_key)
    except s3.exceptions.ClientError as e:
        if e.response['Error']['Code'] == '404':
            print(f"File {file_key} not found in bucket")
            continue
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    row = [source, len(annotations_df), annotations_df['Name'].nunique(), annotations_df['Label ID'].nunique()]
    data2.append(row)
    labelset2 = labelset2 | set(annotations_df['Label ID'].unique())

  0%|          | 0/1468 [00:00<?, ?it/s]/tmp/ipykernel_8613/3951974977.py:17: DtypeWarning: Columns (10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
  2%|▏         | 23/1468 [00:02<01:24, 17.17it/s]

File coralnet-public-images/s1288/annotations.csv not found in bucket


  2%|▏         | 29/1468 [00:02<01:51, 12.90it/s]/tmp/ipykernel_8613/3951974977.py:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
  2%|▏         | 34/1468 [00:03<01:56, 12.34it/s]

File coralnet-public-images/s1388/annotations.csv not found in bucket
File coralnet-public-images/s1514/annotations.csv not found in bucket


  3%|▎         | 45/1468 [00:04<01:56, 12.23it/s]

...

 62%|██████▏   | 915/1468 [01:42<00:32, 16.88it/s]

File coralnet-public-images/s554/annotations.csv not found in bucket


/tmp/ipykernel_8613/3951974977.py:17: DtypeWarning: Columns (2,8,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
 63%|██████▎   | 919/1468 [01:43<01:19,  6.89it/s]/tmp/ipykernel_8613/3951974977.py:17: DtypeWarning: Columns (10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
 81%|████████  | 1189/1468 [02:10<00:24, 11.17it/s]/tmp/ipykernel_8613/3951974977.py:17: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
 83%|████████▎ | 1224/1468 [02:15<00:36,  6.71it/s]/tmp/ipykernel_8613/3951974977.py:17: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
 85%|███

File coralnet-public-images/s7176/annotations.csv not found in bucket


 99%|█████████▉| 1460/1468 [02:42<00:00, 12.13it/s]/tmp/ipykernel_8613/3951974977.py:17: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
/tmp/ipykernel_8613/3951974977.py:17: DtypeWarning: Columns (2,3,4,5,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
100%|██████████| 1468/1468 [02:45<00:00,  8.88it/s]


In [10]:
print(f"The total number of unique labels is {len(labelset2)}")

The total number of unique labels is 3312


In [11]:
coralnet_summary_upd_df = pd.DataFrame(data2, columns=['Source', 'Num Annotations', 'Num Images', 'Num Unique Labels'])
coralnet_summary_upd_df

,Source,Num Annotations,Num Images,Num Unique Labels
0,s1073/,45000,225,27
1,s1076/,41800,209,28
2,s109/,198500,3970,78
3,s1162/,25,5,4
4,s1184/,50,10,7
...,...,...,...,...
1424,s841/,58200,3880,75
1425,s842/,93739,2592,3
1426,s843/,56400,2820,64
1427,s921/,3150,105,27


In [12]:
print("There are a total of %d annotations across %d images (average %d annotations per image) and %d unique labels." %
      (coralnet_summary_upd_df["Num Annotations"].sum(), coralnet_summary_upd_df["Num Images"].sum(), 
       coralnet_summary_upd_df["Num Annotations"].sum() / coralnet_summary_upd_df["Num Images"].sum(), len(labelset2)))

There are a total of 21397835 annotations across 515658 images (average 41 annotations per image) and 3312 unique labels.


In [13]:
coralnet_comparison_df = pd.merge(coralnet_summary_df, coralnet_summary_upd_df, 
                    on='Source', 
                    how='outer',
                    suffixes=('_orig', '_upd'))
coralnet_comparison_df = coralnet_comparison_df.sort_values('Source')
coralnet_comparison_df

,Source,Num Annotations_orig,Num Images_orig,Num Unique Labels_orig,Num Annotations_upd,Num Images_upd,Num Unique Labels_upd
0,s1073/,45000.0,225.0,27.0,45000.0,225.0,27.0
1,s1076/,41800.0,209.0,28.0,41800.0,209.0,28.0
2,s109/,197100.0,3942.0,78.0,198500.0,3970.0,78.0
3,s1097/,16860.0,1686.0,78.0,NaN,NaN,NaN
4,s1162/,25.0,5.0,4.0,25.0,5.0,4.0
...,...,...,...,...,...,...,...
1474,s841/,58200.0,3880.0,75.0,58200.0,3880.0,75.0
1475,s842/,30309.0,940.0,3.0,93739.0,2592.0,3.0
1476,s843/,56400.0,2820.0,64.0,56400.0,2820.0,64.0
1477,s921/,1860.0,62.0,27.0,3150.0,105.0,27.0


In [14]:
coralnet_comparison_df.isna().sum(axis=0)

Source                      0
Num Annotations_orig      905
Num Images_orig           905
Num Unique Labels_orig    905
Num Annotations_upd        50
Num Images_upd             50
Num Unique Labels_upd      50
dtype: int64

In [15]:
coralnet_comparison_df[coralnet_comparison_df["Num Annotations_orig"].isna()]

,Source,Num Annotations_orig,Num Images_orig,Num Unique Labels_orig,Num Annotations_upd,Num Images_upd,Num Unique Labels_upd
6,s1189/,NaN,NaN,NaN,0.0,0.0,0.0
21,s1294/,NaN,NaN,NaN,0.0,0.0,0.0
27,s1356/,NaN,NaN,NaN,0.0,0.0,0.0
38,s1545/,NaN,NaN,NaN,0.0,0.0,0.0
42,s1577/,NaN,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...
1467,s7494/,NaN,NaN,NaN,30.0,1.0,4.0
1468,s7513/,NaN,NaN,NaN,2400.0,96.0,18.0
1469,s7519/,NaN,NaN,NaN,1575.0,63.0,6.0
1470,s7525/,NaN,NaN,NaN,60.0,2.0,4.0


# Check unmapped labels

In [16]:
import requests
def initialize_coralnet_mapping(
    mapping_endpoint="https://api.datamermaid.org/v1/classification/labelmappings/?provider=CoralNet",
):
    response = requests.get(mapping_endpoint)
    data = response.json()
    labelset = data["results"]

    while data["next"]:
        response = requests.get(data["next"])
        data = response.json()
        labelset.extend(data["results"])
    label_mapping = {
        label["provider_id"]: label["benthic_attribute_name"] for label in labelset
    }
    return label_mapping

labelmapping = initialize_coralnet_mapping()

In [17]:
s3 = boto3.client('s3')
bucket_name = 'dev-datamermaid-sm-sources'

labels = []

for source in tqdm.tqdm(folders_new):
    if not source.startswith('s'):
        print(source)

    file_key = f'coralnet-public-images/{source}annotations.csv'

    # Get the file from S3
    try:
        s3.head_object(Bucket=bucket_name, Key=file_key)
    except s3.exceptions.ClientError as e:
        if e.response['Error']['Code'] == '404':
            print(f"File {file_key} not found in bucket")
            continue
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    labels = labels + list(annotations_df['Label ID'])

  0%|          | 0/1468 [00:00<?, ?it/s]/tmp/ipykernel_8613/3393459972.py:20: DtypeWarning: Columns (10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
  1%|▏         | 22/1468 [00:02<01:33, 15.38it/s]

File coralnet-public-images/s1288/annotations.csv not found in bucket


  2%|▏         | 28/1468 [00:02<01:52, 12.85it/s]/tmp/ipykernel_8613/3393459972.py:20: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
  2%|▏         | 34/1468 [00:03<02:08, 11.17it/s]

File coralnet-public-images/s1388/annotations.csv not found in bucket
File coralnet-public-images/s1514/annotations.csv not found in bucket


  3%|▎         | 45/1468 [00:04<01:55, 12.33it/s]

...

 62%|██████▏   | 913/1468 [02:55<02:02,  4.52it/s]

File coralnet-public-images/s554/annotations.csv not found in bucket


 62%|██████▏   | 916/1468 [02:55<01:47,  5.14it/s]/tmp/ipykernel_8613/3393459972.py:20: DtypeWarning: Columns (2,8,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
 63%|██████▎   | 919/1468 [02:57<02:58,  3.08it/s]/tmp/ipykernel_8613/3393459972.py:20: DtypeWarning: Columns (10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
 81%|████████  | 1190/1468 [04:10<01:16,  3.64it/s]/tmp/ipykernel_8613/3393459972.py:20: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
 83%|████████▎ | 1225/1468 [04:22<01:17,  3.15it/s]/tmp/ipykernel_8613/3393459972.py:20: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd

File coralnet-public-images/s7176/annotations.csv not found in bucket


 99%|█████████▉| 1460/1468 [05:42<00:02,  3.18it/s]/tmp/ipykernel_8613/3393459972.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
100%|█████████▉| 1461/1468 [05:42<00:02,  2.73it/s]/tmp/ipykernel_8613/3393459972.py:20: DtypeWarning: Columns (2,3,4,5,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  annotations_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
100%|██████████| 1468/1468 [05:46<00:00,  4.23it/s]


In [18]:
s3 = boto3.client('s3')
bucket_name = 'dev-datamermaid-sm-sources'

for i, source in tqdm.tqdm(enumerate(folders_new)):
    if not source.startswith('s'):
        print(source)

    file_key = f'coralnet-public-images/{source}labelset.csv'

    # Get the file from S3
    try:
        s3.head_object(Bucket=bucket_name, Key=file_key)
    except s3.exceptions.ClientError as e:
        if e.response['Error']['Code'] == '404':
            print(f"File {file_key} not found in bucket")
            continue
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    if i==0:
        labelset_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    else:
        labelset_df = pd.concat([labelset_df, pd.read_csv(io.BytesIO(obj['Body'].read()))], ignore_index=True)

1468it [01:17, 19.02it/s]


In [19]:
labelset_df

,Label ID,Name,Short Code
0,438,Montipora capitata,Moncap
1,2182,Montipora capitata algal overgrowth,Moncap AO
2,2185,Montipora capitata bleached,Moncap BL
3,2184,Montipora capitata tissue loss,Moncap TL
4,439,Montipora flabellata,Monfla
...,...,...,...
73693,3356,Water_H20_RR,WATE**R
73694,3190,algae others,AL
73695,3345,fleshy seawed,FS*
73696,3183,corals:Macroalga,MA**


In [20]:
coralnet_name_map = {k:v for k, v in zip(labelset_df['Label ID'], labelset_df['Name'])}

In [21]:
label_counts = pd.Series(labels).value_counts().reset_index()
label_counts.columns = ['CoralNet ID', 'Count']
label_counts = label_counts.sort_values("Count", ascending=False)
label_counts['CoralNet Name'] = label_counts['CoralNet ID'].map(lambda x: coralnet_name_map.get(x, None))
label_counts['Mermaid Name'] = label_counts['CoralNet ID'].map(lambda x: labelmapping.get(str(x), None))
label_counts

,CoralNet ID,Count,CoralNet Name,Mermaid Name
0,82,3356391,Turf algae,Turf algae
1,84,1745694,Sand,Sand
2,6911,1247926,Not coral,None
3,101,951415,CCA (crustose coralline algae),Crustose coralline algae
4,2513,693971,Pavement,Other
...,...,...,...,...
3268,354,1,Molluscs: Chitons,Other invertebrates
3269,3352,1,Coral Juvenile,None
3270,9056,1,Gelidium latifolium,None
3271,1900,1,ARMS-CREP-Corallimorph,None


In [22]:
total_annotations = label_counts['Count'].sum()
mapped_annotations = label_counts[label_counts['Mermaid Name'].notna()]["Count"].sum()
unmapped_annotations = label_counts[label_counts['Mermaid Name'].isna()]["Count"].sum()

print(f"Currently, there are {label_counts.shape[0]} labels with {total_annotations} unique coral reef annotations.")
print(f"Out of these, {label_counts['Mermaid Name'].notna().sum()} labels with {mapped_annotations} ({mapped_annotations/total_annotations:.2%}) annotations have a label mapping to Mermaid.")
print(f"The remaining, {label_counts['Mermaid Name'].isna().sum()} labels with {unmapped_annotations} ({unmapped_annotations/total_annotations:.2%}) annotations do not have a label mapping to Mermaid.")

Currently, there are 3312 labels with 21397835 unique coral reef annotations.
Out of these, 683 labels with 16466400 (76.95%) annotations have a label mapping to Mermaid.
The remaining, 2629 labels with 4931435 (23.05%) annotations do not have a label mapping to Mermaid.


In [23]:
label_counts[label_counts["Mermaid Name"].isna()]

,CoralNet ID,Count,CoralNet Name,Mermaid Name
2,6911,1247926,Not coral,None
19,1871,178294,ARMS-CREP-CCA,None
20,7462,171024,Blank_tile,None
27,626,125453,PLATE,None
31,5770,112701,Mesh,None
...,...,...,...,...
3267,7260,1,Halophila,None
3269,3352,1,Coral Juvenile,None
3270,9056,1,Gelidium latifolium,None
3271,1900,1,ARMS-CREP-Corallimorph,None


In [24]:
label_counts[(label_counts["Mermaid Name"].isna())*(label_counts["Count"]>2000)]

,CoralNet ID,Count,CoralNet Name,Mermaid Name
2,6911,1247926,Not coral,None
19,1871,178294,ARMS-CREP-CCA,None
20,7462,171024,Blank_tile,None
27,626,125453,PLATE,None
31,5770,112701,Mesh,None
...,...,...,...,...
566,1723,2064,Patella sp.,None
569,7787,2033,Mytilus galloprovincialis,None
571,7576,2015,Botrylloides niger,None
572,7855,2013,Hormosiraceae banksii,None
